## Imports

In [ ]:
import logging
import os
from typing import List, Optional

import hydra
import omegaconf
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from lightning.pytorch import Callback
from omegaconf import DictConfig, ListConfig

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO

# Force the execution of __init__.py if this file is executed directly.
import tvp  # noqa
from tvp.data.datamodule import MetaData
from tvp.data.datasets.registry import get_dataset
from tvp.task_vectors.task_vectors import TaskVector
from tvp.utils.io_utils import load_model_from_artifact
from tvp.utils.utils import build_callbacks
from torch.nn.utils import vector_to_parameters
from torch.nn.utils import parameters_to_vector

import numpy as np

from collections import Counter

pylogger = logging.getLogger(__name__)

torch.set_float32_matmul_precision("high")

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="playground")

In [ ]:
cfg = compose(config_name="task_vectors", overrides=[])

In [ ]:
seed_index_everything(cfg)

cfg.core.tags = enforce_tags(cfg.core.get("tags", None))

template_core: NNTemplateCore = NNTemplateCore(
    restore_cfg=cfg.train.get("restore", None),
)
logger: NNLogger = NNLogger(logging_cfg=cfg.train.logging, cfg=cfg, resume_id=template_core.resume_id)

## Load models

In [ ]:
import copy


zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

finetuned_models = {
    dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
    for dataset in cfg.task_vectors.to_apply
}

zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

## Repair

In [ ]:
import torch.nn as nn


class ResetConv(nn.Module):
    def __init__(self, conv):
        super().__init__()
        self.out_channels = conv.out_channels
        self.conv = conv
        self.bn = nn.BatchNorm2d(self.out_channels)
        self.rescale = False

    def set_stats(self, goal_mean, goal_var, eps=1e-5):
        self.bn.bias.data = goal_mean
        goal_std = (goal_var + eps).sqrt()
        self.bn.weight.data = goal_std

    def forward(self, x):
        x = self.conv(x)
        if self.rescale:
            x = self.bn(x)
        else:
            self.bn(x)
        return x


class ResetLinear(nn.Module):
    def __init__(self, layer):
        # print(f"\n\n[ResetLinear] layer: {layer}\n\n")
        super().__init__()
        self.layer = layer
        self.bn = nn.BatchNorm1d(layer.out_features)
        self.weight = layer.weight
        self.bias = layer.bias

    def set_stats(self, goal_mean, goal_std):
        self.bn.bias.data = goal_mean
        self.bn.weight.data = goal_std

    def forward(self, x):
        # x.shape: [L, N, C] (in torch.nn.BatchNorm1d doc notation)
        x = self.layer(x)

        # match current shape to shape required by BatchNorm1d
        x = x.permute(1, 2, 0)
        # x.shape: [N, C, L] (in torch.nn.BatchNorm1d doc notation)

        x = self.bn(x)

        # match current shape to shape required by Linear
        x = x.permute(2, 0, 1)
        # x.shape: [L, N, C] (in torch.nn.BatchNorm1d doc notation)

        return x


class ResetLayerNorm(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.layer = layer
        self.bn = nn.BatchNorm1d(layer.normalized_shape[0])

    def set_stats(self, goal_mean, goal_std):
        self.bn.bias.data = goal_mean
        self.bn.weight.data = goal_std

    def forward(self, x):
        x = self.layer(x)
        return self.bn(x)


def replace_layers(module):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d):
            setattr(module, name, ResetConv(child))
        # elif isinstance(child, nn.LayerNorm):
        #     setattr(module, name, ResetLayerNorm(child))
        elif isinstance(child, nn.Linear):
            setattr(module, name, ResetLinear(child))
        else:
            replace_layers(child)


def make_tracked_net(model):
    tracked_model = copy.deepcopy(model)
    replace_layers(tracked_model.model.visual)
    return tracked_model.eval()

#### Wrap and compute stats

In [ ]:
dataset = get_dataset(
    cfg.nn.data.train_dataset,
    preprocess_fn=zeroshot_model.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

In [ ]:
from tqdm import tqdm


def reset_bn_stats(model, epochs, loader):
    """
    Reset batchnorm stats. We use the train loader with data augmentation as this gives better results.
    """
    # resetting stats to baseline first as below is necessary for stability
    for m in model.modules():
        if type(m) == nn.BatchNorm2d:
            m.momentum = None  # use simple average
            m.reset_running_stats()

    # run a single train epoch with augmentations to recalc stats
    model.train()
    for _ in range(epochs):
        with torch.no_grad():
            for batch in tqdm(loader):
                if isinstance(batch, Dict):
                    input = batch["x"]
                else:
                    input = batch[0]
                    # print(f"[reset_bn_stats] input.shape before model(): {input.shape}")
                out = model(input.cuda())
                # print(f"[reset_bn_stats] out.shape after model()   : {out.shape}")

In [ ]:
# dataset_name = 'CIFAR100'
dataset_name = cfg.nn.data.train_dataset.replace("Val", "")
finetuned_model = finetuned_models[dataset_name]
tracked_finetuned_model = make_tracked_net(finetuned_models[dataset_name])

In [ ]:
finetuned_model.model.visual

In [ ]:
reset_bn_stats(tracked_finetuned_model.cuda(), 1, dataset.train_loader)

## Task vectors

In [ ]:
tracked_finetuned_model

In [ ]:
flatten = lambda model: parameters_to_vector(model.parameters())

zeroshot_vec = flatten(zeroshot_model)

In [ ]:
task_vectors = [
    TaskVector.from_models(zeroshot_model, finetuned_models[dataset]) for dataset in cfg.task_vectors.to_apply
]

In [ ]:
def apply_task_vector(model, task_vector):
    model.load_state_dict({k: v + task_vector[k] for k, v in model.state_dict().items()})

### Aggregate task vectors

In [ ]:
with torch.no_grad():
    task_vectors = torch.stack(
        [flatten(finetuned_models[dataset]) - zeroshot_vec for dataset in cfg.task_vectors.to_apply]
    )

### Standard task vectors

In [ ]:
task_vectors_sum = torch.sum(task_vectors, dim=0)

In [ ]:
alpha = 0.8

multi_task_vector = task_vectors_sum / len(task_vectors)

In [ ]:
delta_model = copy.deepcopy(zeroshot_model)
vector_to_parameters(multi_task_vector, delta_model.parameters())

In [ ]:
task_equipped_model = copy.deepcopy(zeroshot_model)
apply_task_vector(task_equipped_model, delta_model.state_dict())

In [ ]:
classification_head_identifier = f"{cfg.nn.module.model.model_name}_{cfg.nn.data.dataset.dataset_name}_head"
classification_head = load_model_from_artifact(
    artifact_path=f"{classification_head_identifier}:latest", run=logger.experiment
)

model = hydra.utils.instantiate(
    cfg.nn.module, encoder=task_equipped_model, classifier=classification_head, _recursive_=False
)

## Load dataset

In [ ]:
seed_index_everything(cfg)

dataset = get_dataset(
    cfg.nn.data.train_dataset,
    preprocess_fn=model.encoder.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

storage_dir: str = cfg.core.storage_dir

pylogger.info("Instantiating the <Trainer>")
trainer = pl.Trainer(
    default_root_dir=storage_dir,
    plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
    logger=False,
    callbacks=callbacks,
    **cfg.train.trainer,
)

## Evaluation

In [ ]:
# pylogger.info("Evaluating on the training set")
# trainer.test(model=model, dataloaders=dataset.train_loader)

pylogger.info("Evaluating on the test set!")
trainer.test(model=model, dataloaders=dataset.test_loader)

Keep this accuracy in mind, as it will be used as baseline to compare all upcoming experiments against

# Experiment

## Dataset stats

Compute and export mean and std for every dataset, if not already present on disk.<br>
Otherwhise, load mean and std for each dataset

### Define utility methods

In [ ]:
# Define a function to get the full path of a module
def get_module_path(module, parent_name=""):
    if parent_name:
        return f"{parent_name}.{module._get_name()}"
    return module._get_name()


# Define a recursive function to collect batch norm stats from 'Reset' layers
def collect_bn_stats(model, parent_name=""):
    bn_stats = {}
    for name, module in model.named_children():
        # Construct the full path of the current module
        module_path = f"{parent_name}.{name}" if parent_name else name

        # Check for batch normalization layers within Reset layers
        if isinstance(module, ResetConv) or isinstance(module, ResetLinear):
            for sub_name, sub_module in module.named_children():
                if isinstance(sub_module, nn.BatchNorm2d) or isinstance(sub_module, nn.BatchNorm1d):
                    bn_stats[f"{module_path}.{sub_name}"] = {
                        "running_mean": sub_module.running_mean.clone().detach(),
                        "running_var": sub_module.running_var.clone().detach(),
                        "num_batches_tracked": sub_module.num_batches_tracked.clone().detach(),
                    }
        # Recursively check submodules
        bn_stats.update(collect_bn_stats(module, module_path))

    return bn_stats

### Define objects

In [ ]:
dataset_stats = {}  # S in board pictures

# if "./dataset_stats.pth" is on disk, do not compute stats!
populate_stats = True

### Load models

In [ ]:
if populate_stats:
    # TODO once done, try without it, as all methods use copy.deepcopy, so hygiene should be guaranteed

    import copy

    zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

    zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

    finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

    finetuned_models = {
        dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
        for dataset in cfg.task_vectors.to_apply
    }

    zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

### Stats computation

In [ ]:
if populate_stats:
    for dataset_name in cfg.task_vectors.to_apply:
        print(f"[dataset_name]: {dataset_name}\n")

        # begin load the dataset

        dataset = get_dataset(
            dataset_name=dataset_name,
            preprocess_fn=zeroshot_model.train_preprocess,
            location=cfg.nn.data.data_path,
            batch_size=cfg.nn.data.batch_size.train,
        )

        #   end load the dataset

        finetuned_model = copy.deepcopy(finetuned_models[dataset_name])

        # begin compute layer-wise statistics

        tracked_finetuned_model = make_tracked_net(finetuned_model)

        reset_bn_stats(tracked_finetuned_model.cuda(), 1, dataset.train_loader)

        # end compute layer-wise statistics

        # begin store layer-wise statistics

        dataset_stats[dataset_name] = collect_bn_stats(tracked_finetuned_model.model.visual)

        # end store layer-wise statistics

    torch.save(dataset_stats, "./dataset_stats.pth")

else:
    print(f"Loading dataset stats from './dataset_stats.pth'...")

    dataset_stats = torch.load("./dataset_stats.pth")

## Dataset anchors

### Define objects

In [ ]:
n_anchor_batches = 1

anchors = {}

populate_anchors = False

### Load models

In [ ]:
if populate_anchors:
    # TODO once done, try without it, as all methods use copy.deepcopy, so hygene should be guaranteed

    import copy

    zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

    zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

    finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

    finetuned_models = {
        dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
        for dataset in cfg.task_vectors.to_apply
    }

    zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

### Anchors computations

In [ ]:
if populate_anchors:
    for dataset_name in cfg.task_vectors.to_apply:
        print(f"[dataset_name]: {dataset_name}\n")

        dataset = get_dataset(
            dataset_name=dataset_name,
            preprocess_fn=zeroshot_model.train_preprocess,
            location=cfg.nn.data.data_path,
            batch_size=cfg.nn.data.batch_size.train,
        )

        model: nn.Module = copy.deepcopy(finetuned_models[dataset_name])
        model.eval()

        with torch.no_grad():
            for id, train_batch in enumerate(dataset.train_loader):
                if id == n_anchor_batches:
                    break

                if isinstance(train_batch, Dict):
                    input: torch.Tensor = train_batch["x"]
                else:
                    input: torch.Tensor = train_batch[0]

                anchors[dataset_name] = model.cuda()(input.cuda())

    torch.save(anchors, "./anchors.pth")

else:
    print(f"Loading anchors from './anchors.pth'...")

    anchors = torch.load("./anchors.pth")

In [ ]:
anchors_tensor = torch.stack([anchors[dataset_name] for dataset_name in anchors.keys()])

In [ ]:
anchors_tensor.shape

In [ ]:
list(enumerate((anchors.keys())))

## Evaluation (naive)

### Load models

In [ ]:
# TODO once done, try without it, as all methods use copy.deepcopy, so hygene should be guaranteed

import copy

zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

finetuned_models = {
    dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
    for dataset in cfg.task_vectors.to_apply
}

zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

### Load test dataset

In [ ]:
dataset_name = "GTSRB"

In [ ]:
# seed_index_everything(cfg)

dataset = get_dataset(
    dataset_name=dataset_name,
    preprocess_fn=zeroshot_model.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

storage_dir: str = cfg.core.storage_dir

pylogger.info("Instantiating the <Trainer>")
trainer = pl.Trainer(
    default_root_dir=storage_dir,
    plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
    logger=False,
    callbacks=callbacks,
    **cfg.train.trainer,
)

### Apply all task vectors

In [ ]:
flatten = lambda model: parameters_to_vector(model.parameters())

zeroshot_vec = flatten(zeroshot_model)

In [ ]:
task_vectors = [
    TaskVector.from_models(zeroshot_model, finetuned_models[dataset]) for dataset in cfg.task_vectors.to_apply
]

In [ ]:
with torch.no_grad():
    task_vectors = torch.stack(
        [flatten(finetuned_models[dataset]) - zeroshot_vec for dataset in cfg.task_vectors.to_apply]
    )

In [ ]:
task_vectors_sum = torch.sum(task_vectors, dim=0)

In [ ]:
alpha = 0.8

multi_task_vector = task_vectors_sum / len(task_vectors)

In [ ]:
delta_model = copy.deepcopy(zeroshot_model)
vector_to_parameters(multi_task_vector, delta_model.parameters())

In [ ]:
task_equipped_model = copy.deepcopy(zeroshot_model)
apply_task_vector(task_equipped_model, delta_model.state_dict())

In [ ]:
classification_head_identifier = f"{cfg.nn.module.model.model_name}_{dataset_name}_head"
classification_head = load_model_from_artifact(
    artifact_path=f"{classification_head_identifier}:latest", run=logger.experiment
)

model = hydra.utils.instantiate(
    cfg.nn.module, encoder=task_equipped_model, classifier=classification_head, _recursive_=False
)

### Evaluate (naive)

In [ ]:
pylogger.info("Evaluating on the test set!")
trainer.test(model=model, dataloaders=dataset.test_loader)

## Evaluation (our method)

### Load models

In [ ]:
# TODO once done, try without it, as all methods use copy.deepcopy, so hygene should be guaranteed

import copy

zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

finetuned_models = {
    dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
    for dataset in cfg.task_vectors.to_apply
}

zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

###  Load test dataset

In [ ]:
# seed_index_everything(cfg)

dataset = get_dataset(
    dataset_name=dataset_name,
    preprocess_fn=zeroshot_model.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

storage_dir: str = cfg.core.storage_dir

pylogger.info("Instantiating the <Trainer>")
trainer = pl.Trainer(
    default_root_dir=storage_dir,
    plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
    logger=False,
    callbacks=callbacks,
    **cfg.train.trainer,
)

### Apply all task vectors

In [ ]:
flatten = lambda model: parameters_to_vector(model.parameters())

zeroshot_vec = flatten(zeroshot_model)

In [ ]:
task_vectors = [
    TaskVector.from_models(zeroshot_model, finetuned_models[dataset]) for dataset in cfg.task_vectors.to_apply
]

In [ ]:
with torch.no_grad():
    task_vectors = torch.stack(
        [flatten(finetuned_models[dataset]) - zeroshot_vec for dataset in cfg.task_vectors.to_apply]
    )

In [ ]:
task_vectors_sum = torch.sum(task_vectors, dim=0)

In [ ]:
alpha = 0.8

multi_task_vector = task_vectors_sum / len(task_vectors)

In [ ]:
delta_model = copy.deepcopy(zeroshot_model)
vector_to_parameters(multi_task_vector, delta_model.parameters())

In [ ]:
task_equipped_model = copy.deepcopy(zeroshot_model)
apply_task_vector(task_equipped_model, delta_model.state_dict())

### Prepare model for REPAIR

Keep this here, as the classification head may have layers that may be detected as REPAIRable, even tho we don't want to repair them

In [ ]:
tracked_finetuned_model = make_tracked_net(task_equipped_model)

### Evaluate (our method)

In [ ]:
classification_head_identifier = f"{cfg.nn.module.model.model_name}_{dataset_name}_head"
classification_head = load_model_from_artifact(
    artifact_path=f"{classification_head_identifier}:latest", run=logger.experiment
)

model = hydra.utils.instantiate(
    cfg.nn.module, encoder=tracked_finetuned_model, classifier=classification_head, _recursive_=False
)

In [ ]:
def majority_voting(similarity: torch.Tensor):
    # similarity.shape: [B, M]

    # Get the index of the highest similarity for each element in the batch
    max_similarity_indices = torch.argmax(similarity, dim=1)  # shape: [B]

    # Perform majority voting
    majority_vote = Counter(max_similarity_indices.cpu().numpy()).most_common(1)[0][0]

    return majority_vote

In [ ]:
def centroid_based_similarity(batch: torch.Tensor, anchors_tensor: torch.Tensor):
    # batch.shape: [B, D]
    # anchors_tensor.shape: [M, K, D]

    M, K, D = anchors_tensor.shape
    B = batch.shape[0]

    # Compute centroids of each anchor set
    centroids = anchors_tensor.mean(dim=1)  # shape: [M, D]

    # Compute cosine similarity
    # Normalize batch and centroids
    batch_norm = F.normalize(batch, p=2, dim=1)  # shape: [B, D]
    centroids_norm = F.normalize(centroids, p=2, dim=1)  # shape: [M, D]

    # Compute similarity
    similarity = torch.mm(batch_norm, centroids_norm.t())  # shape: [B, M]

    return similarity

In [ ]:
batch_wise_majority_votes = []

from tqdm import tqdm

# Iterate through the test data loader and compute point-to-set distances
for id, test_batch in tqdm(list(enumerate(dataset.test_loader))):
    with torch.no_grad():
        if isinstance(test_batch, dict):
            input: torch.Tensor = test_batch["x"]
        else:
            input: torch.Tensor = test_batch[0]

        emb = finetuned_models[dataset_name].cuda()(input.cuda())  # [B, D]
        # print(f"[emb]: {emb.shape}")

        similarities = centroid_based_similarity(emb, anchors_tensor)
        # print(f"[similarities]: {similarities.shape}")

        batch_wise_majority_votes.append(majority_voting(similarities))
        # print(f"[majority_vote]: {batch_wise_majority_votes[-1]}")

batch_wise_majority_votes = np.asarray(batch_wise_majority_votes)
most_similar_dataset_id = Counter(batch_wise_majority_votes).most_common(1)[0][0]

In [ ]:
most_similar_dataset_id

In [ ]:
most_similar_dataset = list(anchors.keys())[most_similar_dataset_id]
most_similar_dataset

In [ ]:
most_similar_dataset = get_dataset(
    dataset_name=most_similar_dataset,
    preprocess_fn=model.encoder.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

In [ ]:
model.encoder.model.visual.conv1.bn.running_mean[:10]  # should be all zeroes

In [ ]:
reset_bn_stats(model.cuda(), 1, most_similar_dataset.train_loader)

In [ ]:
model.encoder.model.visual.conv1.bn.running_mean[:10]  # should NOT be all zeroes

In [ ]:
pylogger.info("Evaluating on the test set (stats set via reset_bn_stats()!)")
trainer.test(model=model, dataloaders=dataset.test_loader)

In [ ]:
model.encoder.model.visual.conv1.bn.running_mean = dataset_stats[dataset_name]["conv1.bn"]["running_mean"]
model.encoder.model.visual.conv1.bn.running_var = dataset_stats[dataset_name]["conv1.bn"]["running_var"]

for i in range(12):
    model.encoder.model.visual.transformer.resblocks[i].attn.out_proj.bn.running_mean = dataset_stats[dataset_name][
        f"transformer.resblocks.{i}.attn.out_proj.bn"
    ]["running_mean"]
    model.encoder.model.visual.transformer.resblocks[i].mlp.c_fc.bn.running_mean = dataset_stats[dataset_name][
        f"transformer.resblocks.{i}.mlp.c_fc.bn"
    ]["running_mean"]
    model.encoder.model.visual.transformer.resblocks[i].mlp.c_proj.bn.running_mean = dataset_stats[dataset_name][
        f"transformer.resblocks.{i}.mlp.c_proj.bn"
    ]["running_mean"]

    model.encoder.model.visual.transformer.resblocks[i].attn.out_proj.bn.running_var = dataset_stats[dataset_name][
        f"transformer.resblocks.{i}.attn.out_proj.bn"
    ]["running_var"]
    model.encoder.model.visual.transformer.resblocks[i].mlp.c_fc.bn.running_var = dataset_stats[dataset_name][
        f"transformer.resblocks.{i}.mlp.c_fc.bn"
    ]["running_var"]
    model.encoder.model.visual.transformer.resblocks[i].mlp.c_proj.bn.running_var = dataset_stats[dataset_name][
        f"transformer.resblocks.{i}.mlp.c_proj.bn"
    ]["running_var"]

In [ ]:
pylogger.info("Evaluating on the test set! (stats set via dataset_stats)")
trainer.test(model=model, dataloaders=dataset.test_loader)